In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.io as pio
import plotly.express as px

from ast import literal_eval

try:
    from cfuzzyset import cFuzzySet as FuzzySet
except ImportError:
    from fuzzyset import FuzzySet


In [2]:
pt_df = pd.read_csv("./data/pt_ratings.csv")
salaries_df = pd.read_csv("./data/salaries.csv")
rmp_df = pd.read_csv("./data/rmp_ratings.csv")

In [3]:
pt_df

,courses,average_rating,type,reviews,name,slug
0,"['INST737', 'ENPM808W', 'ENPM808W', 'ENPM808W']",NaN,professor,[],A Seyed,seyed
1,"['MUSC800W', 'MUSC830W', 'MUSC830W']",NaN,professor,[],Aaron Goldman,goldman_aaron
2,"['THET678', 'THET499']",NaN,professor,[],Aaron Posner,posner
3,"['NFSC431', 'NFSC679R', 'NFSC112', 'HLTH672', ...",NaN,professor,[],Abani Pradhan,pradhan
4,"['CMLT235', 'ENGL101', 'ENGL234', 'ENGL101H', ...",NaN,professor,[],Abbey Morgan,morgan_abbey
...,...,...,...,...,...,...
11835,[],2.0,professor,"[{'professor': 'Lance Shapiro', 'course': 'BSC...",Lance Shapiro,shapiro_lance
11836,[],5.0,ta,"[{'professor': 'Livingstone Imonitie', 'course...",Livingstone Imonitie,imonitie
11837,[],5.0,ta,"[{'professor': 'Kathryn Lawless', 'course': 'U...",Kathryn Lawless,lawless_kathryn
11838,[],5.0,ta,"[{'professor': 'Matt Finnan', 'course': 'ENEE1...",Matt Finnan,finnan_matt


In [13]:
pt_review_ex = literal_eval(pt_df.iloc[15].reviews)[0]
pt_review_ex

{'professor': 'Adam Hsieh',
 'course': 'BIOE404',
 'review': "Although it is somewhat hard to stay focused in his class, Hsieh is a good professor. He is a really smart and fair guy, and wants to make sure his students do well in the class. The grading policy he has is amazing and the tests are fairly straightforward. Make sure to make awesome cheat sheets for the tests and then you don't have to worry about the biology. (I wrote in all of the biology notes in size six font on one side of my cheat sheet for each exam, didn't study it besides actually typing it out, and it worked out really well.) Also Hsieh was really approachable if you had any questions and is happy to help.",
 'rating': 4,
 'expected_grade': 'A',
 'created': '2014-12-17T23:49:00Z'}

In [11]:
rmp_review_ex = literal_eval(rmp_df.iloc[rmp_df.reviews.replace("[]", np.nan).first_valid_index()].reviews)[0]
rmp_review_ex

{'professor': 'Paul Allenza',
 'course': 'BIOT645',
 'review': 'Lot of reading, lot of writing, lot of writing discussion posts. Have to do a LOT of research (which is a huge pain), but it pays off.',
 'rating': 4,
 'expected_grade': 'A',
 'created': '2017-04-14 16:20:04 +0000 UTC'}

In [ ]:
def get_fl(s):
    sl = s.split()
    return f"{sl[0]} {sl[-1]}"


salary_fuzz = FuzzySet(salaries_df["name"].unique())
salary_fl_fuzz = FuzzySet(salaries_df["name"].apply(get_fl).unique())

matches_salary_to_pt = {}

l = len(pt_df["name"].unique())
for i, pt_name in enumerate(pt_df["name"].unique()):
    matches = salary_fuzz.get(pt_name)
    match_conf, match_name = matches[0]
    # print(f"[{i + 1}/{l}] {pt_name} -> {matches}")

    if match_conf <= 0.75:
        matches = salary_fl_fuzz.get(pt_name)
        match_conf, match_name = matches[0]
        # print("RETRY")
        # print(f"[{i + 1}/{l}] {pt_name} -> {matches}")

    if match_conf >= 0.75:
        matches_salary_to_pt[match_name] = pt_name

len(matches_salary_to_pt)
